In [ ]:
import pandas as pd
import os

report_path = os.getcwd() + "/reports/Monday-WorkingHours.pcap.report.csv"

In [ ]:
df = pd.read_csv(report_path, index_col="id")

In [ ]:
df["Predictions"].value_counts()

In [ ]:
df["Labels"] = 0

In [ ]:
from sklearn.metrics import auc, precision_recall_curve, accuracy_score, precision_score, recall_score

def score_predictions(y_truth, y_predictions):
    score = {}

    precision, recall, _ = precision_recall_curve(y_truth, y_predictions)
    score["prauc"] = auc(recall, precision)
    score["precision"] = precision_score(y_truth, y_predictions)
    score["recall"] = recall_score(y_truth, y_predictions)
    score["accuracy"] = accuracy_score(y_truth, y_predictions)

    return score

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

In [ ]:
df["Predictions"] = le.fit_transform(df["Predictions"])

In [ ]:
score_predictions(df["Predictions"], df["Labels"])

I spent quite a few hours trying to put together a way to label the netflow generated from the packet capture file, but I was unable to. I think I am missing a key peice in labeling it. I dont have access to the router logs, just captures from an interface that appears to be between the router/firewall and the rest of the network.